In [4]:
# Importar librerías 
import pandas as pd
import os

# Definir función para parsear fechas
def parse_date(date):
    return pd.to_datetime(date, format='%d/%m/%y')

# Establecer directorio de trabajo
os.chdir("/Users/adrianamedrano/Documents/Ariana/Residencias/PrototipoResidencias/DataSets")

# Leer archivo CSV de artículos
df_articulos = pd.read_csv('Articulos.txt', encoding='latin1')

# Inicializar diccionario para almacenar últimas fechas 
fecha_max = {}  

# Leer archivo CSV de movimientos por chunk para optimizar
chunksize = 100000  
for chunk in pd.read_csv('Movinv.txt', chunksize=chunksize, encoding='latin1', parse_dates=['Fecmovinv'], date_parser=parse_date, dtype={15: str}):
    
    # Filtrar chunk por tipos de movimiento relevantes
    filtered_chunk = chunk[chunk['Tipmovinv'].isin([100, 102, 111])]
    
    # Obtener fecha máxima por código de artículo
    df_fechas = filtered_chunk.groupby('Claart')['Fecmovinv'].max().to_dict()
    
    # Actualizar diccionario si fecha es más reciente
    for k, v in df_fechas.items():
        v_str = v.strftime('%d/%m/%y')
        if k not in fecha_max or v_str > fecha_max[k]:
            fecha_max[k] = v_str
            
# Asignar fecha máxima en dataframe de artículos
df_articulos['FecUltven'] = df_articulos['Claart'].map(fecha_max)

In [5]:
# Guardar dataframe
df_articulos.to_csv('/Users/adrianamedrano/Documents/Ariana/Residencias/PrototipoResidencias/DataSets_Codigo/Articulos.txt', index=False, encoding="latin1")